# ACO Hyperparameter Tuning for Graph Coloring Problem

This notebook performs hyperparameter tuning for Ant Colony Optimization (ACO) algorithm applied to the Graph Coloring Problem using Optuna.

## Workflow
1. **Environment Setup**: Detect execution environment (Colab/Local) and configure paths
2. **Parameter Configuration**: Define hyperparameter search space
3. **Optimization**: Run Optuna tuner with objective function
4. **Testing**: Evaluate best parameters on testing dataset
5. **Visualization**: Generate and save performance plots

## 1. Environment Detection and Setup

In [1]:
import sys
import os

# Check if running in Google Colab environment
IS_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IS_COLAB}")

if IS_COLAB:
    print("Colab environment detected. Will mount Google Drive.")
    # Mount Google Drive if running in Colab
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully at /content/drive")
else:
    print("Local environment detected. Using local paths.")

Running in Google Colab: False
Local environment detected. Using local paths.


## 2. Path Configuration

Configure paths for data, studies, results, and figures based on the execution environment.

In [2]:
from pathlib import Path

# Configure base paths based on environment
if IS_COLAB:
    # Update this path to match your Google Drive structure
    BASE_PATH = Path('/content/drive/MyDrive/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'
    # Add code path to system path for imports
    sys.path.insert(0, str(CODE_PATH))
else:
    # Local environment paths
    BASE_PATH = Path('/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3')
    CODE_PATH = BASE_PATH / 'code'

# Define data root path (contains tiny_dataset and main_dataset)
DATA_ROOT = BASE_PATH / 'data'

# Verify paths exist
if not BASE_PATH.exists():
    raise FileNotFoundError(f"Base path does not exist: {BASE_PATH}")
if not DATA_ROOT.exists():
    raise FileNotFoundError(f"Data root does not exist: {DATA_ROOT}")

print(f"Base Path: {BASE_PATH}")
print(f"Code Path: {CODE_PATH}")
print(f"Data Root: {DATA_ROOT}")
print(f"\nPath verification: OK")

Base Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3
Code Path: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code
Data Root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Path verification: OK


## 3. Install Dependencies (Colab Only)

In [3]:
# Install required packages if running in Colab
if IS_COLAB:
    print("Installing required packages...")
    !pip install -q networkx==3.2.1 matplotlib==3.8.2 pandas==2.1.4 numpy==1.26.2 optuna==3.5.0
    print("Packages installed successfully!")

## 4. Import Required Libraries

In [4]:
import multiprocessing
from datetime import datetime
from IPython.display import Image, display

# Import project modules
from dataloader import GraphDataLoader
from optuna_tuner import OptunaACOTuner
from aco_gpc import ACOGraphColoring
from objective_function import aco_objective_function
from results_utils import visualize_testing_results, export_results, print_summary_statistics, print_file_locations

print("All libraries imported successfully!")

/Users/mahdy/projects/meta_graph_coloring_antcol/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All libraries imported successfully!


## 5. Configuration

Define dataset selection, study name, and hyperparameter search space.

In [5]:
# Dataset selection: 'tiny_dataset' for quick testing, 'main_dataset' for full experiments
DATASET_NAME = 'tiny_dataset'  # Change to 'main_dataset' for full tuning

# Study name (will be used for study files and result organization)
STUDY_NAME = f'aco_study_{DATASET_NAME}_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

# Number of Optuna trials for hyperparameter tuning
N_TRIALS = 5

# ACO verbose setting 
ACO_VERBOSE = False   # Set to True to see detailed ACO progress

# Hyperparameter search space configuration (parameters to optimize)
PARAM_CONFIG = {
    'iterations': {
        'type': 'int',
        'low': 50,
        'high': 200,
    },
    'num_colors': {
        'type': 'int',
        'low': 20,
        'high': 50,
    },
    'alpha': {
        'type': 'float',
        'low': 0.5,
        'high': 3.0,
    },
    'beta': {
        'type': 'float',
        'low': 1.0,
        'high': 5.0,
    },
    'rho': {
        'type': 'float',
        'low': 0.01,
        'high': 0.5,
    },
    'ant_count': {
        'type': 'int',
        'low': 10,
        'high': 50,
    },
    'Q': {
        'type': 'float',
        'low': 0.1,
        'high': 10.0,
    }
}

print(f"Configuration:")
print(f"  Dataset: {DATASET_NAME}")
print(f"  Study Name: {STUDY_NAME}")
print(f"  Optuna Trials: {N_TRIALS}")
print(f"\nHyperparameters to Optimize:")
for param_name, param_spec in PARAM_CONFIG.items():
    print(f"  {param_name}: [{param_spec['low']}, {param_spec['high']}] ({param_spec['type']})")

Configuration:
  Dataset: tiny_dataset
  Study Name: aco_study_tiny_dataset_20251129_170506
  Optuna Trials: 5

Hyperparameters to Optimize:
  iterations: [50, 200] (int)
  num_colors: [20, 50] (int)
  alpha: [0.5, 3.0] (float)
  beta: [1.0, 5.0] (float)
  rho: [0.01, 0.5] (float)
  ant_count: [10, 50] (int)
  Q: [0.1, 10.0] (float)


## 6. Run Hyperparameter Optimization

Run Optuna optimization using the objective function from separate script.

In [6]:
# Initialize the Optuna tuner
tuner = OptunaACOTuner(
    study_name=STUDY_NAME,
    data_root=str(DATA_ROOT),
    direction='minimize'  # We want to minimize the number of colors used
)

print(f"Tuner initialized with study: {STUDY_NAME}")
print(f"Data root: {DATA_ROOT}")

# Load tuning dataset once (before optimization)
print("\nLoading tuning dataset...")
data_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)
tuning_graphs = data_loader.load_tuning_dataset()
print(f"Loaded {len(tuning_graphs)} graphs for tuning\n")

# Wrapper function to pass tuning graphs to objective function
def objective_wrapper(trial, params, **kwargs):
    return aco_objective_function(
        trial=trial,
        params=params,
        tuning_graphs=tuning_graphs,
        aco_class=ACOGraphColoring,
        verbose=ACO_VERBOSE
    )

print("Objective function wrapper ready!")

Tuner initialized with study: aco_study_tiny_dataset_20251129_170506
Data root: /Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/data

Loading tuning dataset...

Loading Tuning Dataset: tiny_dataset
  gc_20_9:
    Nodes: 22, Edges: 165
    Density: 0.7143, Connected: False
  gc_4_1:
    Nodes: 5, Edges: 4
    Density: 0.4000, Connected: True
  gc_50_9:
    Nodes: 52, Edges: 1104
    Density: 0.8326, Connected: False

Loaded 3 graphs for tuning

Objective function wrapper ready!


/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py:59: FutureWarning: JournalFileStorage has been deprecated in v4.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v4.0.0. Use :class:`~optuna.storages.journal.JournalFileBackend` instead.
  self.storage = JournalStorage(JournalFileStorage(str(self.journal_file)))


## 7. Run Optimization

In [7]:
# Run the hyperparameter optimization
print(f"\nStarting hyperparameter optimization with {N_TRIALS} trials...")
print("=" * 70)

best_params = tuner.optimize(
    objective_func=objective_wrapper,
    param_config=PARAM_CONFIG,
    aco_class=ACOGraphColoring,
    n_trials=N_TRIALS
)

print("\n" + "=" * 70)
print("Optimization completed!")
print("\nBest Parameters Found:")
for param_name, param_value in best_params.items():
    print(f"  {param_name}: {param_value}")

[I 2025-11-29 17:05:06,926] A new study created in Journal with name: aco_study_tiny_dataset_20251129_170506



Starting hyperparameter optimization with 5 trials...
Created new study 'aco_study_tiny_dataset_20251129_170506'
Completed trials: 0/5
Remaining trials: 5

Starting optimization with 5 trials...


  0%|          | 0/5 [00:00<?, ?it/s]

[W 2025-11-29 17:05:06,941] Trial 0 failed with parameters: {'iterations': 125, 'num_colors': 28, 'alpha': 1.93146941708432, 'beta': 1.9396633117117479, 'rho': 0.14429455679687583, 'ant_count': 23, 'Q': 6.315065977804529} because of the following error: AttributeError("'OptunaACOTuner' object has no attribute 'data_loader'").
Traceback (most recent call last):
  File "/Users/mahdy/projects/meta_graph_coloring_antcol/venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/Users/mahdy/projects/meta_graph_coloring_antcol/assignemnt3/code/optuna_tuner.py", line 214, in wrapped_objective
    data_loader=self.data_loader,
                ^^^^^^^^^^^^^^^^
AttributeError: 'OptunaACOTuner' object has no attribute 'data_loader'
[W 2025-11-29 17:05:06,943] Trial 0 failed with value None.


AttributeError: 'OptunaACOTuner' object has no attribute 'data_loader'

## 8. Generate Optimization Visualizations

Generate and display plots showing the optimization history, parameter importances, and parallel coordinates.

In [ ]:
# Generate all optimization plots
print("Generating optimization plots...")
tuner.generate_plots(recreate=True)
print(f"Plots saved to: {DATA_ROOT / 'figures'}")

## 9. Test Best Parameters on Testing Dataset

Evaluate the best parameters on the testing dataset.

In [ ]:
# Initialize data loader for testing
test_loader = GraphDataLoader(str(DATA_ROOT), DATASET_NAME)

# Store testing results
testing_results = {}

print("\nEvaluating best parameters on testing dataset...")
print("=" * 70)


# Test on each graph in the testing dataset
for graph_name, graph in test_loader.load_testing_dataset():
    print(f"\nTesting on graph: {graph_name}")
    
    # Create ACO instance with best parameters and double iterations for thorough testing
    test_params = best_params.copy()
    test_params['iterations'] = int(test_params['iterations'] * 2)
    test_params['verbose'] = True
    
    aco = ACOGraphColoring(graph=graph, **test_params)
    
    # Run ACO optimization
    result = aco.run()
    
    # Store results
    testing_results[graph_name] = {
        'color_count': result['color_count'],
        'conflict_count': result['conflict_count'],
        'iterations_used': result['iterations'],
        'best_solution': result['best_solution']
    }
    
    print(f"  Final Colors: {result['color_count']}")
    print(f"  Final Conflicts: {result['conflict_count']}")

print("Testing completed!")
print("\n" + "=" * 70)

In [ ]:
# Export results and generate visualizations
exported_files = export_results(testing_results, best_params, STUDY_NAME, DATA_ROOT)
figure_path = visualize_testing_results(testing_results, STUDY_NAME, DATA_ROOT)

print(f"\nResults exported successfully!")
print(f"Testing figure saved to: {figure_path}")

## 10. Display Results

Show the testing results figure and summary statistics.

In [ ]:
# Display the testing results figure
display(Image(filename=str(figure_path)))

# Print summary statistics
print_summary_statistics(exported_files['summary_df'])

# Print file locations
print_file_locations(STUDY_NAME, DATA_ROOT, exported_files, figure_path)